# Corresponding

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import numpy
import scipy.signal
import matplotlib.pyplot
import matplotlib.cm
import pathlib
import PIL.Image

In [ ]:
matplotlib.pyplot.rcParams.update({'font.size': 22})

In [ ]:
# Open image and convert to grayscale
path: pathlib.Path = pathlib.Path('./images/tsukuba/scene1.row3.col1.ppm')
img = numpy.array(PIL.Image.open(str(path)).convert('L'))

figure = matplotlib.pyplot.figure(figsize=(10, 10))

matplotlib.pyplot.axis('off')
# matplotlib.pyplot.title('Image I')
_ = matplotlib.pyplot.imshow(img, cmap=matplotlib.cm.gray)

## Write function to detect interst points

In [ ]:
# import eagle.points.response

# def find_interest_points(
#     img: numpy.ndarray, 
#     size_neigh: int, 
#     epsilon: float
# ) -> numpy.ndarray:
#     r = eagle.points.response.kitchen_rosenfeld(img)
#     rp = eagle.points.response.remove_non_maxima(r, size_neigh)
#     points = eagle.points.response.select_with_threshold(rp, epsilon)
#     return points

## Find interset points and compute differentials invariants

In [ ]:
from eagle.points.response import \
    (
        kitchen_rosenfeld,
        remove_non_maxima,
        select_with_threshold
    )

from eagle.points.descriptor.differential import DifferentialInvariant

class ImageFeatures:

    def __init__(self, img: numpy.ndarray):

        self.img = img

        # Find interest points
        r = kitchen_rosenfeld(self.img)
        rp = remove_non_maxima(r, size_neigh=3)
        self.interest_points = select_with_threshold(rp, epsilon=0.3)

        # self.nb_points = self.interest_points.shape[1]
        
        # Compute differential invaraint vector
        self.differential_invariant = DifferentialInvariant(self.img)

In [ ]:
imgs: list[numpy.ndarray] = []

img_features: list[ImageFeatures] = []

for img in imgs:
    img_features.append(ImageFeatures(img))

## Find corresponding

### Compute scores between differential invaraints

In [ ]:
def symmetric_indices(indices: tuple[int, int]) -> tuple[int, int]:
    i, j = indices
    return i, j if j >= i else j, i

In [ ]:
import eagle.points.descriptor.distance

def compute_score(feature_1: ImageFeatures, feature_2: ImageFeatures) -> numpy.ndarray:

    """ For each interest point from first feature, we compare interest point \n
    from second feature with the distance between differential invariant vector
    """
    
    nb_pts_1 = feature_1.interest_points.shape[0]
    nb_pts_2 = feature_2.interest_points.shape[0]

    score = numpy.zeros(shape=(nb_pts_1, nb_pts_2))

   

    for i in range(0, nb_pts_1):

        pt_i = tuple(feature_1.interest_points[i])
        vec_i = feature_1.differential_invariant[pt_i]
        
        for j in range(0, nb_pts_2):

            pt_j = tuple(feature_1.interest_points[j])
            vec_j = feature_1.differential_invariant[pt_j]
            distance = eagle.points.descriptor.distance.euclidean(vec_i, vec_j)

            score[i, j] = distance

    return score

In [ ]:
nb_imgs = len(imgs)

# Sym Matrix : idx_i, idx_j = i, j if j <= i else j, i
scores = numpy.zeros(shape=(nb_imgs, nb_imgs), dtype=numpy.ndarray)

for i in range(0, nb_imgs):
    features_i = img_features[i]
    for j in range(i+1, nb_imgs):
        features_j = img_features[i]
        scores[symmetric_indices((i, j))] = compute_score(features_i, features_j)        

### Find best corresponding

In [ ]:
nb_imgs = len(imgs)

EPSILON_DISTANCE = 0.001


# Find j such as we have a maximum of corresponding
# for i
corresp = list[tuple[int, numpy.ndarray]] = []

# corresp[i] index interest pts between i and j
corresp_img = numpy.empty(shape=nb_imgs, dtype=int)

corresp_pts = numpy.empty(shape=nb_imgs, dtype=numpy.ndarray)

for i in range(0, nb_imgs):

    best_pts: numpy.ndarray = corresp_pts
    
    for j in range(i+1, nb_imgs):
        
        score_ij: numpy.ndarray = scores[symmetric_indices(i, j)]
        tmp = numpy.transpose(numpy.array(numpy.where(score_ij <= EPSILON_DISTANCE)))

        if (best_pts is None) or (best_pts.shape[0] < tmp.shape[0]):
            corresp_img[i] = j
            corresp_pts[i] = best_pts

## Compute homography

In [ ]:
import numpy.linalg

def hartley_normaliation(points: numpy.ndarray) -> numpy.ndarray:

    n = points.shape[0]
    mean = numpy.mean(points, axis=0)
    center_points = points-mean

    norms = numpy.sqrt(numpy.sum(center_points**2, axis=1))
    a = numpy.sqrt(2) / numpy.mean(norms)
    # a = numpy.sqrt(2) / ((1 / n) * numpy.sum(norms))

    u, v = points[:, 0], points[:, 1]
    u_mean, v_mean = mean[:, 0], mean[:, 1]

    for i in range(0, n):
        u_hartley = a*u - a*u_mean
        v_hartley = a*v - a*v_mean

    points = numpy.array([u_hartley, v_hartley])

    return points


def homography_estimation(points_1: numpy.ndarray, points_2: numpy.ndarray, normalisation: bool = False) -> numpy.ndarray:

    
    pts1, pts2 = \
        hartley_normaliation(numpy.copy(points_1)), hartley_normaliation(numpy.copy(points_2)) \
            if normalisation else points_1, points_2


    nb_points = pts1.shape[0]

    u, v, up, vp = pts1[:, 0], pts1[:, 1], pts2[:, 0], pts2[:, 1]

    D = numpy.zeros(shape=(2*nb_points, 9))

    for i in range(0, nb_points, 2):
        
        u, v = pts1[i]
        up, vp = pts2[i]

        D[i:i+1, :] = numpy.array(
            [
                [ u[i], v[i], 1, 0, 0, 0, -up[i]*u[i], -up[i]*v[i], -up[i] ],
                [ 0, 0, 0, u[i], v[i], 1, -vp[i]*u[i], -vp[i]*v[i], -vp[i] ]
            ]
        )

    eigen_values, eigen_vectors = numpy.linalg.eig(D.T @ D)
    indmin = numpy.argmin(eigen_values)
    v = eigen_vectors[:, indmin]

    h = numpy.reshape(v, newshape=(3, 3))

    return h